# Inbound Notebook

This notebook is designed to semi-automate the reporting process for the Inbound team. It will streamline data extraction, transformation, and loading into a pre-formatted Excel file.

## Manual Preparation

The first step involves manually preparing the data in Excel:

1. **Filter the Pivot Table:**
   - Apply filters to the pivot table to extract the following categories:
     - Active
     - Canceled
     - Pending Signature
     - Net

2. **Create Separate Sheets:**
   - For each category (Active, Canceled, Pending Signature, Net), create a separate sheet in the Excel file containing the filtered data.

3. **Save the Excel File:**
   - Save the prepared Excel file with a specific name, ensuring it contains the sheets with the filtered data.

4. **Upload the Excel File:**
   - Upload the prepared Excel file to the designated directory.

## Library Installation

Ensure that the necessary libraries are installed before running the notebook.

In [ ]:
# Import necessary libraries
%pip install openpyxl
from openpyxl import load_workbook
import pandas as pd
import os
import re
import itertools

print("Skeleton setup complete!")

## Variable Declaration

Set the variables for file paths, sheet names, and other configurations. Update these variables for each specific project.

In [ ]:
# Path to the Excel file (change this for each project)
excel_file_path = '/workspaces/Finetwork-Automation/inbound/Metabase.xlsx'
csv_file_path = '/workspaces/Finetwork-Automation/inbound/Informe de métricas históricas.csv'

# Sheet names for different categories
sheet_active = 'ACTIVOS'
sheet_canceled = 'CANCELADOS'
sheet_pending = 'PTE DE FIRMA'

# Range to read (change this for each project)
start_row = 8
end_row = 65
usecols = 'A:AF'

print("Variables defined correctly!")

## Extract Data from "Informe de métricas históricas" CSV file
Extract data from the CSV file and convert it directly to a DataFrame.

In [ ]:
df = pd.read_csv(csv_file_path)

pivot_df = df.pivot_table(index=['Agent', 'StartInterval', 'EndInterval'], 
                          values=['Contacts handled incoming', 'Contacts transferred out internal'], 
                          aggfunc='first') 


print(pivot_df)

## Extract Data from 'Active' Sheet

Extract data from the "Active" sheet within the specified range and convert it directly to a DataFrame.

## Verify Columns in 'Active' Sheet

Verify the number of columns in the "Active" sheet to ensure the range is within bounds.

In [ ]:
# Function to verify the number of columns
def verify_columns(file_path, sheet_name):
    workbook = load_workbook(filename=file_path, data_only=True)
    sheet = workbook[sheet_name]
    max_column = sheet.max_column
    return max_column

# Check the number of columns in the 'Active' sheet
max_column_active = verify_columns(excel_file_path, 'ACTIVOS')
print(f"Max column in 'Active' sheet: {max_column_active}")

# Check if the number of columns matches the expected range
expected_columns = 32  # Columns from A to AF (inclusive)
if max_column_active < expected_columns:
    usecols = f"A:{chr(64+max_column_active)}"
    print(f"Adjusted usecols to: {usecols}")
else:
    print(f"Using default usecols: {usecols}")

## Extract Data from 'Active' Sheet

Extract data from the "Active" sheet within the specified range and convert it directly to a DataFrame.

In [ ]:
def load_sheet_as_dataframe(file_path, sheet_name, start_row, end_row, usecols):
    # Load data from the specified sheet and range into a DataFrame
    df = pd.read_excel(file_path, sheet_name=sheet_name, usecols=usecols, skiprows=start_row-1, nrows=end_row-start_row+1)
    print(f"Data from '{sheet_name}' sheet loaded successfully.")
    return df

# Extract data from 'Active' sheet
active_df = load_sheet_as_dataframe(excel_file_path, 'ACTIVOS', start_row, end_row, usecols)

## Extract Data from 'Canceled' Sheet

Extract data from the "Canceled" sheet within the specified range and convert it directly to a DataFrame.

In [ ]:
# Extract data from 'Canceled' sheet
canceled_df = load_sheet_as_dataframe(excel_file_path, 'CANCELADOS', start_row, end_row, usecols)

## Extract Data from 'Pending Signature' Sheet

Extract data from the "Pending Signature" sheet within the specified range and convert it directly to a DataFrame.

In [ ]:
# Extract data from 'Pending Signature' sheet
pending_signature_df = load_sheet_as_dataframe(excel_file_path, 'PTE FIRMA', start_row, end_row, usecols)

## Display DataFrames

Display the first few rows of each DataFrame to verify the data.

In [ ]:
# Display the DataFrames
print("Active DataFrame:")
display(active_df.head())

print("Canceled DataFrame:")
display(canceled_df.head())

print("Pending Signature DataFrame:")
display(pending_signature_df.head())

## Replace NaN with 0

Replace all NaN values in the DataFrames with 0 to facilitate further transformations.

In [ ]:
def replace_nan_with_zero(df):
    """
    Replace all NaN values in the DataFrame with 0.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to process.
    
    Returns:
    pd.DataFrame: The processed DataFrame with NaN replaced by 0.
    """
    df = df.fillna(0)
    print("Replaced NaN with 0.")
    return df

## Apply Transformation

Apply the transformation to replace NaN values with 0 in each DataFrame.

In [ ]:
# Apply the transformation
active_df = replace_nan_with_zero(active_df)
canceled_df = replace_nan_with_zero(canceled_df)
pending_signature_df = replace_nan_with_zero(pending_signature_df)

# Display the transformed DataFrames
print("Active DataFrame after replacing NaN:")
display(active_df.head())

print("Canceled DataFrame after replacing NaN:")
display(canceled_df.head())

print("Pending Signature DataFrame after replacing NaN:")
display(pending_signature_df.head())

## Load Agents List

Load the list of all agents from the "Agents" sheet.

In [ ]:
# Load the list of agents
agents_df = pd.read_excel(excel_file_path, sheet_name='AGENTES', usecols='A')
agents_list = agents_df.iloc[:, 0].tolist()
print("Agents list loaded successfully!")
print(agents_list)

## Reorganize and Group Data in Informe.csv

Transform the CSV data into a more structured format by pivoting the DataFrame. This step groups the data by agent and date, allowing for easy access to each agent's daily call statistics. The resulting structure organizes call data by date for each agent, creating a clear and manageable dataset for further processing.


In [ ]:
import pandas as pd

# Cargar el CSV en un DataFrame
df = pd.read_csv(csv_file_path)

# Crear una tabla pivote que agrupe por 'Agent', 'StartInterval', 'EndInterval'
pivot_df = df.pivot_table(index=['Agent', 'StartInterval', 'EndInterval'], 
                          values=['Contacts handled incoming', 'Contacts transferred out internal'], 
                          aggfunc='first')  # Utiliza el primer valor si hay duplicados

# Mostrar el DataFrame pivotado
print(pivot_df)

## Verify and Complete Data

Verify that all agents are present in each DataFrame. If an agent is missing, add a row with zeros for that agent.

In [ ]:
def ensure_all_agents(df, agents_list):
    """
    Ensure all agents are present in the DataFrame. Add missing agents with zero values and remove agents not in the list.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to check and update.
    agents_list (list): The list of all agents.
    
    Returns:
    pd.DataFrame: The updated DataFrame with all agents.
    """
    # Get the list of agents in the DataFrame
    existing_agents = df.iloc[:, 0].tolist()
    
    # Find missing agents
    missing_agents = [agent for agent in agents_list if agent not in existing_agents]
    
    # Add rows for missing agents with zero values
    for agent in missing_agents:
        zero_row = pd.DataFrame([[agent] + [0] * (df.shape[1] - 1)], columns=df.columns)
        df = pd.concat([df, zero_row], ignore_index=True)
    
    # Remove agents not in the agents list
    df = df[df.iloc[:, 0].isin(agents_list)]
    
    print(f"Added {len(missing_agents)} missing agents and removed {len(existing_agents) - len(df)} agents not in the list.")
    return df

# Apply the function to each DataFrame
active_df = ensure_all_agents(active_df, agents_list)
canceled_df = ensure_all_agents(canceled_df, agents_list)
pending_signature_df = ensure_all_agents(pending_signature_df, agents_list)

# Display the updated DataFrames
print("Active DataFrame after ensuring all agents:")
display(active_df.head())

print("Canceled DataFrame after ensuring all agents:")
display(canceled_df.head())

print("Pending Signature DataFrame after ensuring all agents:")
display(pending_signature_df.head())

## Verify and Complete Pivot Table Data

Verify that all agents are present in the Pivot Table. If an agent is missing, add a row with zeros for that agent.

### Remove extra agents

In [ ]:
import pandas as pd

def remove_extra_agents(pivot_df, agents_list):
    """
    Remove agents from the pivot DataFrame that are not in the agents_list.
    
    Parameters:
    pivot_df (pd.DataFrame): The pivoted DataFrame to update.
    agents_list (list): The list of all agents to keep.
    
    Returns:
    pd.DataFrame: The updated pivot DataFrame with only the agents in agents_list.
    """
    # Verificar si 'Agent' es un nivel de índice, si no, restaurarlo
    if 'Agent' not in pivot_df.index.names:
        pivot_df.index.names = ['Agent', 'StartInterval', 'EndInterval']
    
    # Obtener la cantidad inicial de agentes
    initial_agents = len(pivot_df.index.get_level_values('Agent').unique())

    # Filtrar el DataFrame para mantener solo los agentes que están en agents_list
    pivot_df = pivot_df[pivot_df.index.get_level_values('Agent').isin(agents_list)]
    
    # Obtener la cantidad final de agentes
    final_agents = len(pivot_df.index.get_level_values('Agent').unique())

    # Calcular cuántos agentes se han eliminado
    agents_removed = initial_agents - final_agents

    print(f"Removed {agents_removed} agents. Remaining agents: {final_agents}")
    return pivot_df

def add_missing_agents(pivot_df, agents_list):
    """
    Add missing agents to the pivot DataFrame with zero values for relevant columns.
    
    Parameters:
    pivot_df (pd.DataFrame): The pivoted DataFrame to update.
    agents_list (list): The list of all agents to ensure are present.
    
    Returns:
    pd.DataFrame: The updated pivot DataFrame with all agents.
    """
    # Verificar si 'Agent' es un nivel de índice, si no, restaurarlo
    if 'Agent' not in pivot_df.index.names:
        pivot_df.index.names = ['Agent', 'StartInterval', 'EndInterval']

    # Obtener la lista de agentes presentes en el DataFrame pivoteado
    existing_agents = pivot_df.index.get_level_values('Agent').unique()

    # Identificar agentes faltantes que están en la lista pero no en el pivot_df
    missing_agents = [agent for agent in agents_list if agent not in existing_agents]

    # Crear combinaciones de 'StartInterval' y 'EndInterval'
    date_combinations = list(itertools.product(
        pivot_df.index.get_level_values('StartInterval').unique(),
        pivot_df.index.get_level_values('EndInterval').unique()
    ))

    # Añadir filas para los agentes faltantes con valores cero
    for agent in missing_agents:
        for start_interval, end_interval in date_combinations:
            zero_row = pd.Series(
                [0, 0],  # Valores para 'Contacts handled incoming' y 'Contacts transferred out internal'
                index=pivot_df.columns,
                name=(agent, start_interval, end_interval)
            )
            pivot_df = pd.concat([pivot_df, zero_row.to_frame().T])

    print(f"Added {len(missing_agents)} missing agents.")
    return pivot_df

# Aplicar las funciones a la tabla pivoteada
informe_df = remove_extra_agents(pivot_df, agents_list)
informe_df = add_missing_agents(informe_df, agents_list)

# Mostrar el DataFrame actualizado
print("Informe DataFrame after ensuring all agents:")
display(informe_df.head(55))


### Add extra agents

In [ ]:
def add_missing_agents(pivot_df, agents_list):
    """
    Add missing agents to the pivot DataFrame with zero values for relevant columns.
    
    Parameters:
    pivot_df (pd.DataFrame): The pivoted DataFrame to update.
    agents_list (list): The list of all agents to ensure are present.
    
    Returns:
    pd.DataFrame: The updated pivot DataFrame with all agents.
    """
    # Verificar si 'Agent' es un nivel de índice, si no, restaurarlo
    if 'Agent' not in pivot_df.index.names:
        pivot_df.index.names = ['Agent', 'StartInterval', 'EndInterval']

    # Obtener la lista de agentes presentes en el DataFrame pivoteado
    existing_agents = pivot_df.index.get_level_values('Agent').unique()

    # Identificar agentes faltantes que están en la lista pero no en el pivot_df
    missing_agents = [agent for agent in agents_list if agent not in existing_agents]

    # Crear combinaciones de 'StartInterval' y 'EndInterval'
    date_combinations = list(itertools.product(
        pivot_df.index.get_level_values('StartInterval').unique(),
        pivot_df.index.get_level_values('EndInterval').unique()
    ))

    # Añadir filas para los agentes faltantes con valores cero
    for agent in missing_agents:
        for start_interval, end_interval in date_combinations:
            zero_row = pd.Series(
                [0, 0],  # Valores para 'Contacts handled incoming' y 'Contacts transferred out internal'
                index=pivot_df.columns,
                name=(agent, start_interval, end_interval)
            )
            pivot_df = pd.concat([pivot_df, zero_row.to_frame().T])

    print(f"Added {len(missing_agents)} missing agents.")
    return pivot_df

# Aplicar las funciones a la tabla pivoteada
informe_df = remove_extra_agents(pivot_df, agents_list)
informe_df = add_missing_agents(informe_df, agents_list)

# Mostrar el DataFrame actualizado
print("Informe DataFrame after ensuring all agents:")
display(informe_df.head(55))

## Assign Values to Emails

Assign numerical values to each email and add them as a new column in the DataFrames.

In [ ]:
# Dictionary mapping emails to their respective values
email_values = {
    'albaaraujo@originaltelecom.es': 1,
    'albertocanto@originaltelecom.es': 2,
    'albertosanchez@originaltelecom.es': 3,
    'anasanchez@originaltelecom.es': 4,
    'antonio.reina@originaltelecom.es': 5,
    'azahara.garcia@originaltelecom.es': 6,
    'beatriz.gomez@originaltelecom.es': 7,
    'carmen.cornejo@originaltelecom.es': 8,
    'carolinafuentes@originaltelecom.es': 9,
    'cesar.arnaldo@originaltelecom.es': 10,
    'david.molero@originaltelecom.es': 11,
    'elenaborrero@originaltelecom.es': 12,
    'estefania.panea@originaltelecom.es': 13,
    'francisco.perdomo@originaltelecom.es': 14,
    'gonzalofalcon@originaltelecom.es': 15,
    'guillermo.hurtado@originaltelecom.es': 16,
    'irati.izaguirre@originaltelecom.es': 17,
    'ivan.barroso@originaltelecom.es': 18,
    'laura.eguens@originaltelecom.es': 19,
    'lailasetati@originaltelecom.es': 20,
    'leonor.lopez@originaltelecom.es': 21,
    'dolores.cortes@originaltelecom.es': 22,
    'manuelvaldes@originaltelecom.es': 23,
    'manuelventura@originaltelecom.es': 24,
    'mar.aguila@originaltelecom.es': 25,
    'mariangeles.bueso@originaltelecom.es': 26,
    'mariaarroyo@originaltelecom.es': 27,
    'maria.torres@originaltelecom.es': 28,
    'marta.dorado@originaltelecom.es': 29,
    'mauricio.pozo@originaltelecom.es': 30,
    'miguel.segura@originaltelecom.es': 31,
    'miriam.rodriguez@originaltelecom.es': 32,
    'mar.marchena@originaltelecom.es': 33,
    'natividad.sanchez@originaltelecom.es': 34,
    'nereacerezo@originaltelecom.es': 35,
    'oscar.rivilla@originaltelecom.es': 36,
    'patricia.rios@originaltelecom.es': 37,
    'paulavilla@originaltelecom.es': 38,
    'pilar.deval@originaltelecom.es': 39,
    'sara.elkhelyfy@originaltelecom.es': 40,
    'sergio.vazquez@originaltelecom.es': 41,
    'yicel.patricia@originaltelecom.es': 42,
    'yzabelly.gomes@originaltelecom.es': 43
}

# Add a new column to each DataFrame with the email values
def add_email_values(df, email_values):
    df['email_value'] = df.iloc[:, 0].map(email_values)
    return df

# Apply the function to each DataFrame
active_df = add_email_values(active_df, email_values)
canceled_df = add_email_values(canceled_df, email_values)
pending_signature_df = add_email_values(pending_signature_df, email_values)

# Display the updated DataFrames with the new 'email_value' column
print("Active DataFrame with email values:")
display(active_df.head())

print("Canceled DataFrame with email values:")
display(canceled_df.head())

print("Pending Signature DataFrame with email values:")
display(pending_signature_df.head())

## Sort DataFrames by Email Values

Sort the DataFrames based on the numerical values assigned to the emails.

In [ ]:
# Sort each DataFrame by the 'email_value' column
def sort_by_email_value(df):
    df = df.sort_values(by='email_value')
    return df

# Apply the sorting function to each DataFrame
active_df = sort_by_email_value(active_df)
canceled_df = sort_by_email_value(canceled_df)
pending_signature_df = sort_by_email_value(pending_signature_df)

# Display the sorted DataFrames
print("Sorted Active DataFrame:")
display(active_df.head())  # Displaying first 20 rows for testing

print("Sorted Canceled DataFrame:")
display(canceled_df.head())  # Displaying first 20 rows for testing

print("Sorted Pending Signature DataFrame:")
display(pending_signature_df.head())  # Displaying first 20 rows for testing

## Remove 'email_value' Column

After sorting the DataFrames based on the email values, the 'email_value' column should be removed to prevent interference with further calculations.

In [ ]:
# Function to remove the 'email_value' column
def remove_email_value_column(df):
    if 'email_value' in df.columns:
        df = df.drop(columns=['email_value'])
    return df

# Apply the function to each DataFrame
active_df = remove_email_value_column(active_df)
canceled_df = remove_email_value_column(canceled_df)
pending_signature_df = remove_email_value_column(pending_signature_df)

# Display the updated DataFrames without the 'email_value' column
print("Active DataFrame after removing 'email_value' column:")
display(active_df.head())

print("Canceled DataFrame after removing 'email_value' column:")
display(canceled_df.head())

print("Pending Signature DataFrame after removing 'email_value' column:")
display(pending_signature_df.head())

column_values = active_df.iloc[:, 0].tolist()
print("Values from the second column of active_df:")
print(column_values)

## Remove agents from the three sheets. Later will have to be adequated in the previous piece of code.

In [ ]:
# Lista de agentes a eliminar
agents_to_remove = ['mar.aguila@originaltelecom.es', 'formacion4@originaltelecom.es', 'formacion10@originaltelecom.es', 'formacion3@originaltelecom.es']

# Eliminar filas donde la primera columna contiene alguno de los agentes en la lista
active_df = active_df[~active_df.iloc[:, 0].isin(agents_to_remove)]
canceled_df = canceled_df[~canceled_df.iloc[:, 0].isin(agents_to_remove)]
pending_signature_df = pending_signature_df[~pending_signature_df.iloc[:, 0].isin(agents_to_remove)]

# Mostrar el DataFrame actualizado
print("Active DataFrame after removing specific agents:")
display(active_df.head())

column_values = active_df.iloc[:, 0].tolist()
print("Values from the second column of active_df:")
print(column_values)

## Process Active DataFrame

Sum all numeric values in each row, divide the result by 2, and store the final values.

In [ ]:
# Calculate sum of numeric values in each row, divided by 2
active_sums = active_df.iloc[:, 1:].sum(axis=1) / 2  # Assuming the first column is not numeric
print("Calculated sums for 'Active' DataFrame:")
print(active_sums.head())

## Update finalFile Excel Sheet

Update the "finalFile" Excel sheet with the calculated values from the Active DataFrame.

In [ ]:
from openpyxl import load_workbook

# Path to the final file
final_file_path ='/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx'
sheet_name = 'GLOBAL AGOSTO'  # Change this to the correct sheet name

def update_final_file(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column F (6th column)
    start_row = 3
    column = 10  # Column J
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}'.")

# Update the final file with the calculated sums
update_final_file(final_file_path, sheet_name, active_sums)

## Process Canceled DataFrame

Sum all numeric values in each row, divide the result by 2, and store the final values in column N starting from row 3.

In [ ]:
# Calculate sum of numeric values in each row, divided by 2 for 'Canceled' DataFrame
canceled_sums = canceled_df.iloc[:, 1:].sum(axis=1) / 2  # Assuming the first column is not numeric
print("Calculated sums for 'Canceled' DataFrame:")
print(canceled_sums.head())

## Update finalFile Excel Sheet with Canceled Data

Update the "finalFile" Excel sheet with the calculated values from the Canceled DataFrame in column N.

In [ ]:
def update_final_file_canceled(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column N (14th column)
    start_row = 3
    column = 11  # Column K
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}' with Canceled data.")

# Update the final file with the calculated sums for Canceled
update_final_file_canceled(final_file_path, sheet_name, canceled_sums)

## Process Pending Signature DataFrame

Sum all numeric values in each row, divide the result by 2, and store the final values in column Q starting from row 3.

In [ ]:
# Calculate sum of numeric values in each row, divided by 2 for 'Pending Signature' DataFrame
pending_signature_sums = pending_signature_df.iloc[:, 1:].sum(axis=1) / 2  # Assuming the first column is not numeric
print("Calculated sums for 'Pending Signature' DataFrame:")
print(pending_signature_sums.head())

## Update finalFile Excel Sheet with Pending Signature Data

Update the "finalFile" Excel sheet with the calculated values from the Pending Signature DataFrame in column Q.

In [ ]:
def update_final_file_pending_signature(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column Q (17th column)
    start_row = 3
    column = 12  # Column L
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}' with Pending Signature data.")

# Update the final file with the calculated sums for Pending Signature
update_final_file_pending_signature(final_file_path, sheet_name, pending_signature_sums)

## Update finalFile Excel Sheet with Informe Data

Update the "finalFile" Excel sheet with the raw values from the Informe DataFrame in column Q.

## Define Useful Columns for Active DataFrame

Determine the range of columns that will be used for the Active DataFrame.

In [ ]:
# Determine the range of columns for Active DataFrame
active_usecols = active_df.columns[:-1]  # Exclude the last column
print("Active DataFrame Useful Columns:")
print(active_usecols)

## Define Useful Columns for Canceled DataFrame

Determine the range of columns that will be used for the Canceled DataFrame.

In [ ]:
# Determine the range of columns for Canceled DataFrame
canceled_usecols = canceled_df.columns[:-1]  # Exclude the last column
print("Canceled DataFrame Useful Columns:")
print(canceled_usecols)

## Define Useful Columns for Pending Signature DataFrame

Determine the range of columns that will be used for the Pending Signature DataFrame.

In [ ]:
# Determine the range of columns for Pending Signature DataFrame
pending_signature_usecols = pending_signature_df.columns[:-1]  # Exclude the last column
print("Pending Signature DataFrame Useful Columns:")
print(pending_signature_usecols)

## Update Diario Agosto Sheet with Active DataFrame

For each useful column in the Active DataFrame, update the corresponding column in the "DIARIO AGOSTO" sheet.

In [ ]:
def update_diario_agosto_with_last_column(df, file_path):
    """
    Update the "DIARIO AGOSTO" sheet with values from the last column of the Active DataFrame.
    
    Parameters:
    df (pd.DataFrame): The Active DataFrame.
    file_path (str): Path to the Excel file.
    """
    workbook = load_workbook(filename=file_path)
    sheet = workbook['DIARIO AGOSTO']
    
    # Identify the last and penultimate columns in the DataFrame
    last_column_name = df.columns[-2]
    penultimate_column_index = len(df.columns) - 2  # Get the index of the penultimate column
    
    # Define the column mappings: Active DataFrame -> DIARIO AGOSTO
    column_mappings = {
        2: 'C',   # Assuming 'B' is the first column
        3: 'H',   # Assuming 'C' is the second column
        4: 'M',   # Assuming 'D' is the third column
        5: 'R',   # Assuming 'E' is the fourth column
        6: 'AB',  # Assuming 'F' is the fifth column
        7: 'AG',
        8: 'AL',
        9: 'AQ',
        10: 'AV',
        11: 'BA',
        12: 'BF',
        13: 'BK',
        14: 'BP',
        15: 'BU',
        16: 'BZ',
        17: 'CE',
        18: 'CJ',
        19: 'CO',
        20: 'CT',
        21: 'CY',
        22: 'DD',
        23: 'DI',
        24: 'DN',
        25: 'DS',
        26: 'DX',
        27: 'EC',
        28: 'EH',
        29: 'EM',
        30: 'ER',
        31: 'EW'
    }
    
    # Determine the target column based on the penultimate column index
    target_column = column_mappings.get(penultimate_column_index, 'EW')  # Default to 'EW' if index not found
    
    # Update the Excel sheet with values from the last column of the DataFrame
    for row_idx, value in enumerate(df[last_column_name], start=4):  # Assuming Excel updates start at row 4
        sheet[f'{target_column}{row_idx}'] = value
    
    workbook.save(file_path)
    print(f"Updated DIARIO AGOSTO sheet with the last column '{last_column_name}' from Active DataFrame.")

# Update the Diario Agosto sheet with the last column of the Active DataFrame
update_diario_agosto_with_last_column(active_df, final_file_path)

## Update Diario Agosto Sheet with Canceled DataFrame

For each useful column in the Canceled DataFrame, update the corresponding column in the "DIARIO AGOSTO" sheet.

In [ ]:
def update_diario_agosto_with_last_column_canceled(df, file_path):
    """
    Update the "DIARIO AGOSTO" sheet with values from the last column of the Canceled DataFrame.
    
    Parameters:
    df (pd.DataFrame): The Canceled DataFrame.
    file_path (str): Path to the Excel file.
    """
    workbook = load_workbook(filename=file_path)
    sheet = workbook['DIARIO AGOSTO']
    
    # Identify the last and penultimate columns in the DataFrame
    last_column_name = df.columns[-2]
    penultimate_column_index = len(df.columns) - 2  # Get the index of the penultimate column
    
    # Define the column mappings: Canceled DataFrame -> DIARIO AGOSTO
    column_mappings = {
        2: 'D',   
        3: 'I',   
        4: 'N',   
        5: 'S',   
        6: 'AC',  
        7: 'AH',
        8: 'AM',
        9: 'AR',
        10: 'AW',
        11: 'BB',
        12: 'BG',
        13: 'BL',
        14: 'BQ',
        15: 'BV',
        16: 'CA',
        17: 'CF',
        18: 'CK',
        19: 'CP',
        20: 'CU',
        21: 'CZ',
        22: 'DE',
        23: 'DJ',
        24: 'DO',
        25: 'DT',
        26: 'DY',
        27: 'ED',
        28: 'EI',
        29: 'EN',
        30: 'ES',
        31: 'EX'
    }
    
    # Determine the target column based on the penultimate column index
    target_column = column_mappings.get(penultimate_column_index, 'EW')  # Default to 'EW' if index not found
    
    # Update the Excel sheet with values from the last column of the DataFrame
    for row_idx, value in enumerate(df[last_column_name], start=4):  # Assuming Excel updates start at row 4
        sheet[f'{target_column}{row_idx}'] = value
    
    workbook.save(file_path)
    print(f"Updated DIARIO AGOSTO sheet with the last column '{last_column_name}' from Canceled DataFrame.")

# Update the Diario Agosto sheet with the last column of the Canceled DataFrame
update_diario_agosto_with_last_column_canceled(canceled_df, final_file_path)

## Update Diario Agosto Sheet with Pending Signature DataFrame

For each useful column in the Pending Signature DataFrame, update the corresponding column in the "DIARIO AGOSTO" sheet.

In [ ]:
def update_diario_agosto_with_last_column_pending_signature(df, file_path):
    """
    Update the "DIARIO AGOSTO" sheet with values from the last column of the Pending Signature DataFrame.
    
    Parameters:
    df (pd.DataFrame): The Pending Signature DataFrame.
    file_path (str): Path to the Excel file.
    """
    workbook = load_workbook(filename=file_path)
    sheet = workbook['DIARIO AGOSTO']
    
    # Identify the last and penultimate columns in the DataFrame
    last_column_name = df.columns[-2]
    penultimate_column_index = len(df.columns) - 2  # Get the index of the penultimate column
    
    # Define the column mappings: Pending Signature DataFrame -> DIARIO AGOSTO
    column_mappings = {
        2: 'E',   # Adjusted to the next column
        3: 'J',   
        4: 'O',   
        5: 'T',   
        6: 'AD',  
        7: 'AI',
        8: 'AN',
        9: 'AS',
        10: 'AX',
        11: 'BC',
        12: 'BH',
        13: 'BM',
        14: 'BR',
        15: 'BW',
        16: 'CB',
        17: 'CG',
        18: 'CL',
        19: 'CQ',
        20: 'CV',
        21: 'DA',
        22: 'DF',
        23: 'DK',
        24: 'DP',
        25: 'DU',
        26: 'DZ',
        27: 'EE',
        28: 'EJ',
        29: 'EO',
        30: 'ET',
        31: 'EY'
    }
    
    # Determine the target column based on the penultimate column index
    target_column = column_mappings.get(penultimate_column_index, 'EY')  # Default to 'EY' if index not found
    
    # Update the Excel sheet with values from the last column of the DataFrame
    for row_idx, value in enumerate(df[last_column_name], start=4):  # Assuming Excel updates start at row 4
        sheet[f'{target_column}{row_idx}'] = value
    
    workbook.save(file_path)
    print(f"Updated DIARIO AGOSTO sheet with the last column '{last_column_name}' from Pending Signature DataFrame.")

# Update the Diario Agosto sheet with the last column of the Pending Signature DataFrame
update_diario_agosto_with_last_column_pending_signature(pending_signature_df, final_file_path)

## Update Diario Agosto Sheet with Informe DataFrame

For each useful column in the Informe DataFrame, update the corresponding column in the "DIARIO AGOSTO" sheet.